# 🎵 Music Converter Backend on Colab

此笔记本用于在 Google Colab 的免费 GPU 上运行 Music Converter 后端。

**前置准备：**
1. 拥有一个 [Ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) 账号并获取 Authtoken。

In [ ]:
#  1. 初始化环境与获取代码
import os

# 检查 GPU
!nvidia-smi

# === 配置区域 ===
REPO_URL = "https://github.com/dieWehmut/music-converter.git"
BRANCH = "main"
# ================

# 克隆代码
if not os.path.exists("music-converter"):
    !git clone -b $BRANCH $REPO_URL music-converter
    %cd music-converter
else:
    %cd music-converter
    !git pull

print("✅ 代码准备就绪")

In [ ]:
# 2. 安装系统与 Python 依赖 (耗时约 2-3 分钟)

# 1. 安装系统级音频库
!apt-get update -qq
!apt-get install -y libsndfile1 ffmpeg

# 2. 安装 Python 库
# 使用清华源可能在 Colab 较慢，Colab 默认源通常很快
!pip install -r backend/requirements.txt
!pip install pyngrok uvicorn

print("✅ 依赖安装完毕")

In [ ]:
# @title 3. 启动后端服务
import os
import threading
import time
import subprocess
from pyngrok import ngrok

# ==========================================
# 🔐 请在这里填入你的 Ngrok Token
# 格式通常是: 2Qxxxx... (很长的一串字符)
# 不要保留 "<" 或 ">" 符号，只要双引号里的内容
NGROK_AUTHTOKEN = "这里粘贴你的Token" 
# ==========================================

# 1. 设置环境变量
os.environ["MC_ENABLE_LONG_AUDIO"] = "1"
os.environ["MC_DEV_MODE"] = "0"

# 2. 直接设置 Token (删除之前的判断逻辑)
print(f"正在配置 Ngrok Token: {NGROK_AUTHTOKEN[:5]}...") # 打印前5位确认
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 3. 开启隧道
# 终止旧的隧道
ngrok.kill()

try:
    # 建立 HTTP 隧道指向 8000 端口
    public_url = ngrok.connect(8000).public_url
    print(f"\n🚀 ========================================================")
    print(f"🚀 服务已启动！请复制下面的 Public URL 到前端：")
    print(f"👉 {public_url}")
    print(f"📄 API 文档地址: {public_url}/docs")
    print(f"🚀 ========================================================\n")
except Exception as e:
    print(f"❌ Ngrok 连接失败: {e}")
    print("提示: 请检查 Token 是否正确，或者去 Ngrok 官网重置 Token。")

# 4. 启动 Uvicorn 后端
cmd = ["python", "-m", "backend.server"]

try:
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    # 实时打印日志
    print("正在启动后端服务 (Backend)...")
    for line in process.stdout:
        print(line, end="")
        
except KeyboardInterrupt:
    print("🛑 服务已停止")
    ngrok.kill()